In [2]:
from syft_rds.orchestra import setup_rds_stack
from syft_rds import RDS_NOTEBOOKS_PATH

In [3]:
stack = setup_rds_stack(log_level="INFO")

2025-03-06 10:17:01.075 | WARNING  | syft_rds.orchestra:setup_rds_stack:89 - Using shared data directory for both clients. Any file permission checks will be skipped as both clients have access to the same files.
2025-03-06 10:17:01.086 | INFO     | syft_rds.orchestra:setup_rds_stack:112 - Launching mock RDS stack in /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_ctvy9jsu...


In [4]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a dataset as DO

In [9]:
private_path = RDS_NOTEBOOKS_PATH / "data" / "dataset-1" / "private"
mock_path = RDS_NOTEBOOKS_PATH / "data" / "dataset-1" / "mock"

private_path.mkdir(parents=True, exist_ok=True)
mock_path.mkdir(parents=True, exist_ok=True)

with open(RDS_NOTEBOOKS_PATH /"data/dataset-1/private/data.csv", "w") as f:
    f.write("-1,-2,-3")

with open(RDS_NOTEBOOKS_PATH / "data/dataset-1/mock/data.csv", "w") as f:
    f.write("1,2,3")


In [10]:
data = do_client.dataset.create(
    name="dataset-1",  # MUST BE UNIQUE. Throw Exception if already exist.
    path=private_path,  # MUST EXIST
    mock_path=mock_path,
)

In [12]:
do_client.datasets

[Dataset
   uid: d732ea50-7a7c-40d9-9642-a990db8788af
   created_at: 2025-03-06T09:18:23.923585Z
   updated_at: 2025-03-06T09:18:23.923595Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

# Use dataset as DS

In [13]:
ds_client.datasets

[Dataset
   uid: d732ea50-7a7c-40d9-9642-a990db8788af
   created_at: 2025-03-06T09:18:23.923585Z
   updated_at: 2025-03-06T09:18:23.923595Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

In [14]:
ds_client.dataset.get("dataset-1").get_mock_path()


PosixPath('/private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_ctvy9jsu/shared_client_dir/datasites/data_owner@test.openmined.org/public/datasets/dataset-1')

In [18]:
ds_client.dataset.get("dataset-1").get_private_path()

PosixPath('/private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_ctvy9jsu/shared_client_dir/datasites/data_owner@test.openmined.org/private/datasets/dataset-1')

In [21]:
dataset = ds_client.dataset.get("dataset-1")
dataset.describe()

└── data.csv


True

In [29]:
import pathlib
from syft_rds.models.models import Dataset
from syft_rds.utils.render_code_template import render_data_access_template

In [33]:
my_file = RDS_NOTEBOOKS_PATH / "data" / "code1.py"

In [37]:
%%writefile {my_file}

import os

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

print("Hello, world!")

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write("ABC")


Writing /Users/koen/workspace/rds/notebooks/data/code1.py


In [38]:
job = ds_client.jobs.submit(
    user_code_path=my_file,
    dataset_name = dataset.name
)
job


Job
  uid: dd4521c1-cbe4-4363-a0eb-dfb009936ce4
  created_at: 2025-03-06T09:44:51.108507Z
  updated_at: 2025-03-06T09:44:51.108510Z
  name: robust_server_a1yh
  description: None
  runtime: python
  user_code_id: f1c93094-73e1-4193-9c38-09277297ccbd
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/dd4521c1-cbe4-4363-a0eb-dfb009936ce4
  dataset_name: dataset-1

In [39]:
ds_client.run(job)

╭─────────────────────────────────────────────── Job Configuration ───────────────────────────────────────────────╮
│ Starting job                                                                                                    │
│ Execution: python code1.py                                                                                      │
│ Dataset:                                                                                                        │
│ /private/var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/rds_ctvy9jsu/shared_client_dir/datasites/data_owner@te │
│ st.openmined.org/public/datasets/dataset-1 → /data                                                              │
│ Output:   robust_server_a1yh/output → /output                                                                   │
│ Timeout:  1s                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Running job...

Output()

Hello, world!

Job completed successfully!

0